In [6]:
!python --version
import tensorflow as tf
import pandas as pd
print(tf.__version__)
import keras
from sklearn.model_selection import train_test_split

Python 3.10.112.15.0


In [11]:
import numpy as np
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

# Note that Keras should only be imported after the backend
# has been configured. The backend cannot be changed once the
# package is imported.
import keras

In [12]:
# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

11490434/11490434 [==============================] - 1s 0us/step
x_train shape: (60000, 28, 28, 1)
y_train shape: (60000,)
60000 train samples
10000 test samples


In [13]:
# Model parameters
num_classes = 10
input_shape = (28, 28, 1)

model = keras.Sequential(
    [
        keras.layers.Input(shape=input_shape),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.Conv2D(128, kernel_size=(3, 3), activation="relu"),
        keras.layers.GlobalAveragePooling2D(),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(num_classes, activation="softmax"),
    ]
)

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 64)        640       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 10, 10, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 8, 8, 128)         147584    
                                                                 
 global_average_pooling2d (  (None, 128)               0         
 GlobalAveragePooling2D)                                

In [15]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="acc"),
    ],
)


In [ ]:
batch_size = 128
epochs = 20

callbacks = [
    keras.callbacks.ModelCheckpoint(filepath="model_at_epoch_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=2),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.15,
    callbacks=callbacks,
)
score = model.evaluate(x_test, y_test, verbose=0)

Epoch 1/20
399/399 [==============================] - 123s 304ms/step - loss: 0.8236 - acc: 0.7186 - val_loss: 0.1641 - val_acc: 0.9547
Epoch 2/20
399/399 [==============================] - 146s 366ms/step - loss: 0.2364 - acc: 0.9290 - val_loss: 0.0829 - val_acc: 0.9773
Epoch 3/20
399/399 [==============================] - 197s 494ms/step - loss: 0.1637 - acc: 0.9512 - val_loss: 0.0657 - val_acc: 0.9832
Epoch 4/20
399/399 [==============================] - 196s 489ms/step - loss: 0.1276 - acc: 0.9626 - val_loss: 0.0552 - val_acc: 0.9847
Epoch 5/20
399/399 [==============================] - 155s 388ms/step - loss: 0.1084 - acc: 0.9684 - val_loss: 0.0441 - val_acc: 0.9889
Epoch 6/20
 17/399 [>.............................] - ETA: 2:13 - loss: 0.1049 - acc: 0.9651

## OUR ANN!

In [19]:
data = pd.read_csv("training_data/test-training-data.csv")

Y = data['Y']
X = data.drop(['Y'],axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.34, random_state=10)

Y_train = Y_train / 27
Y_test = Y_test / 27

# Model parameters
num_classes = 1
inputs = keras.Input(shape=(294,), name="input")
dense = keras.layers.Dense(64, activation="relu")
x = dense(inputs)
x = keras.layers.Dense(64, activation="relu")(x)
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs=inputs, outputs=outputs, name="first_test_model")
model.summary()

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(X_train, Y_train, epochs=5, validation_split=0.2)

test_scores = model.evaluate(X_test, Y_test, verbose=2)


print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

Model: "first_test_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 294)]             0         
                                                                 
 dense_30 (Dense)            (None, 64)                18880     
                                                                 
 dense_31 (Dense)            (None, 64)                4160      
                                                                 
 dense_32 (Dense)            (None, 1)                 65        
                                                                 
Total params: 23105 (90.25 KB)
Trainable params: 23105 (90.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/5
858/858 [==============================] - 5s 4ms/step - loss: 0.0000e+00 - accuracy: 0.2258 - val_loss: 0.0000e+00 - val_accuracy: 0.22